# Contenido
## Descripción de la base de datos
* Origen / fuentes
* Supervisado o no supervisado
* Descripción
* Rangos
* ouliers
* num, nom, ord
## Objetivo del análisis
## Metodología.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
nombres=[]
marcas=[]
presentaciones = []
precios =[]
lugar = []
for i in range(1,149):
  url = 'https://losprecios.co/%C3%A9xito_t6/despensa?p={0}'
  response = requests.get(url.format(i))

  soup = BeautifulSoup(response.text, 'html.parser')
  conts = soup.find_all(class_='bq-cn-r')
  for cont in conts:
    cont_text = cont.find(class_='mrg-0')
    # print(cont_text)
    cont_nombre = cont_text.find(class_='eít-enl-p')
    nombre = cont_nombre.text
    marca = cont_nombre.find_next('span').find_next('span').text
    presentacion = cont.find(class_='no-lín').text
    precio = cont.find(class_='t-ed-pr').text
    nombres.append(nombre)
    marcas.append(marca)
    presentaciones.append(presentacion)
    precios.append(precio)
    lugar.append('éxito')

df = pd.DataFrame({
'nombres':nombres,
'marcas':marcas,
'presentaciones':presentaciones,
'precios':precios,
'lugar':lugar
})


In [3]:
df

,nombres,marcas,presentaciones,precios,lugar
0,Crema de Leche Dietética,Alquería,225 g,2 656 $,éxito
1,Crema de Leche,Alquería,225 g,2 528 $,éxito
2,Crema de Leche,Nestlé,186 g,4 680 $,éxito
3,Crema de Leche,Alquería,200 g,2 200 $,éxito
4,Crema de Leche,Nestlé,150 g,1 958 $,éxito
...,...,...,...,...,...
1467,Base para Gulash,Maggi,45 g,3 170 $,éxito
1468,Salsa Hoisin,Lee Kum Kee,240 g,11 470 $,éxito
1469,Salsa Provenzal,Casino,420 g,9 240 $,éxito
1470,Sopa Minestrone Preparada,Campbell's,305 g,13 117 $,éxito


In [9]:
url = 'https://losprecios.co/d1_t1/despensa?p={0}'
nombres=[]
marcas=[]
presentaciones = []
precios =[]
lugar = []
for i in range(1,19):
    response = requests.get(url.format(i))
    soup = BeautifulSoup(response.text, 'html.parser')

    conts = soup.find_all(class_ = 'bq-cn-r')
    for cont in conts:
        cont_text = cont.find(class_='mrg-0')
        # print(cont_text)
        cont_nombre = cont_text.find(class_='eít-enl-p')
        nombre = cont_nombre.text
        marca = cont_nombre.find_next('span').find_next('span').text
        presentacion = cont.find(class_='no-lín').text
        precio = cont.find(class_='t-ed-pr').text
        nombres.append(nombre)
        marcas.append(marca)
        presentaciones.append(presentacion)
        precios.append(precio)
        lugar.append('D1')

df2 =  pd.DataFrame({
'nombres':nombres,
'marcas':marcas,
'presentaciones':presentaciones,
'precios':precios,
'lugar':lugar
})



In [10]:
df2

,nombres,marcas,presentaciones,precios,lugar
0,Crema de Leche,Latti,200 g,1 190 $,D1
1,Aceite Vegetal,Gota de Oro,1 000 ml,9 590 $,D1
2,Aceite de Girasol,Don Olio,1 000 ml,5 800 $,D1
3,Aceite de Oliva Extra Virgen,Castell de Ferro,500 ml,16 990 $,D1
4,Aceite de Canola,Don Olio,1 000 ml,14 850 $,D1
...,...,...,...,...,...
175,Paté de Cerdo,Lourisierra,125 g,3 200 $,D1
176,"Arvejas, Zanahorias y Maíz en Lata",Cooltivo,300 g,1 950 $,D1
177,Mezcla para Hamburguesa Vegetariana,Natri,250 g,6 950 $,D1
178,Quinua Preparada,Kitchen & Love,225 g,6 690 $,D1


In [12]:
url = 'https://losprecios.co/ara_t2/despensa?p={0}'
nombres=[]
marcas=[]
presentaciones = []
precios =[]
lugar = []
for i in range(1,14):
    response = requests.get(url.format(i))
    soup = BeautifulSoup(response.text, 'html.parser')

    conts = soup.find_all(class_ = 'bq-cn-r')
    for cont in conts:
        cont_text = cont.find(class_='mrg-0')
        # print(cont_text)
        cont_nombre = cont_text.find(class_='eít-enl-p')
        nombre = cont_nombre.text
        marca = cont_nombre.find_next('span').find_next('span').text
        presentacion = cont.find(class_='no-lín').text
        precio = cont.find(class_='t-ed-pr').text
        nombres.append(nombre)
        marcas.append(marca)
        presentaciones.append(presentacion)
        precios.append(precio)
        lugar.append('Ara')

df3 =  pd.DataFrame({
'nombres':nombres,
'marcas':marcas,
'presentaciones':presentaciones,
'precios':precios,
'lugar':lugar
})

df3


In [37]:
url = 'https://losprecios.co/jumbo_t7/despensa?p={0}'
nombres=[]
marcas=[]
presentaciones = []
precios =[]
lugar = []
for i in range(1,52):
    response = requests.get(url.format(i))
    soup = BeautifulSoup(response.text, 'html.parser')

    conts = soup.find_all(class_ = 'bq-cn-r')
    for cont in conts:
        cont_text = cont.find(class_='mrg-0')
        # print(cont_text)
        cont_nombre = cont_text.find(class_='eít-enl-p')
        nombre = cont_nombre.text
        marca = cont_nombre.find_next('span').find_next('span').text
        presentacion = cont.find(class_='no-lín').text
        precio = cont.find(class_='t-ed-pr').text
        nombres.append(nombre)
        marcas.append(marca)
        presentaciones.append(presentacion)
        precios.append(precio)
        lugar.append('Jumbo')

df4 =  pd.DataFrame({
'nombres':nombres,
'marcas':marcas,
'presentaciones':presentaciones,
'precios':precios,
'lugar':lugar
})

In [66]:
import spacy
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
def extraer_tipo_producto(texto):
    palabras = word_tokenize(texto, language='spanish')
    etiquetado = nltk.pos_tag(palabras)
    for palabra, etiqueta in etiquetado:
        if etiqueta in ['NN', 'NNS', 'NNP', 'NNPS']:  # Sustantivos
            return palabra
    return None

extraer_tipo_producto(text)

'Crema'

In [217]:
df_precios = pd.concat([df,df2,df3,df4])
df_indxed = df_precios.reset_index().drop('index',axis=1)

df_typed = df_indxed.copy()
df_typed["tipo"] = [extraer_tipo_producto(word) for word in df_indxed["nombres"]]
df_typed["precios"] = df_typed["precios"].replace(["\u202f"," "],"", regex=True).apply(lambda x: x.split("$")[0]).astype(float)
# df_typed["presentaciones"] = df_typed["presentaciones"].replace(["\u202f"," "],"", regex=True).apply(lambda x: x.split("g")[0])
# df_typed["presentaciones"] = df_typed["presentaciones"].replace(["\u202f"," "],"", regex=True).apply(lambda x: x.split("ml")[0])
# df_typed["presentaciones"] = df_typed["presentaciones"].replace(["\u202f"," "],"", regex=True).apply(lambda x: x.split("unidades")[0])
# df_typed["presentaciones"] = df_typed["presentaciones"].replace(["\u202f"," "],"", regex=True).apply(lambda x: float(x.split("k")[0]))
presentaciones = []
unidades = []
for i in df_typed["presentaciones"]:
    if i.__contains__('kg'):
        i = i.replace('kg','')
        i = i.replace("\u202f","")
        i = i.replace(" ","")
        i = float(i)*1000
        unidades.append("g")
        presentaciones.append(i)
    elif i.__contains__('ml'):
        i = i.replace('ml','')
        i = i.replace("\u202f","")
        i = i.replace(" ","")
        unidades.append("ml")
        presentaciones.append(i)
    elif i.__contains__('unidades'):
        i = i.replace('unidades','')
        i = i.replace("\u202f","")
        i = i.replace(" ","")
        unidades.append("unidades")
        presentaciones.append(i)
    elif i.__contains__('g'):
        i = i.replace('g','')
        i = i.replace("\u202f","")
        i = i.replace(" ","")
        unidades.append("g")
        presentaciones.append(i)

df_typed["presentaciones"] = presentaciones
df_typed["unidades"] = unidades


In [218]:
df_typed["marcas"][df_indxed["marcas"].str.contains('del ')] = df_typed["marcas"][df_indxed["marcas"].str.contains('del ')].str.replace('del ','')

/tmp/ipykernel_152122/277072489.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_typed["marcas"][df_indxed["marcas"].str.contains('del ')] = df_typed["marcas"][df_indxed["marcas"].str.contains('del ')].str.replace('del ','')


In [229]:
df_typed[df_typed['marcas'].str.contains('Doria')]

,nombres,marcas,presentaciones,precios,lugar,tipo,unidades
272,Espaguetis,Doria,1000,4280.0,éxito,Espaguetis,g
278,Espaguetis,Doria,500,3220.0,éxito,Espaguetis,g
280,Espaguetis Integrales,Doria,500,4300.0,éxito,Espaguetis,g
281,Espaguetis,Doria,250,1910.0,éxito,Espaguetis,g
294,Espaguetis,Doria,400,2512.0,éxito,Espaguetis,g
296,Espaguetis,Doria,250,1528.0,éxito,Espaguetis,g
298,Espaguetis,Doria,250,1680.0,éxito,Espaguetis,g
300,Espaguetis en Tubos,Doria,250,1700.0,éxito,Espaguetis,g
302,Espaguetis,Doria,500,4440.0,éxito,Espaguetis,g
341,Salsa de Tomate,Doria,200,3070.0,éxito,Salsa,g


In [239]:
df_typed[df_typed["marcas"].str.contains('Doria')][df_typed["tipo"].str.contains('Pasta','Macarrones')]


/tmp/ipykernel_152122/4045651307.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_typed[df_typed["marcas"].str.contains('Doria')][df_typed["tipo"].str.contains('Pasta','Macarrones')]


,nombres,marcas,presentaciones,precios,lugar,tipo,unidades
640,Pasta para Lasaña,Doria,250,9010.0,éxito,Pasta,g
1042,Pasta en Conchas,Doria,250,1600.0,éxito,Pasta,g
1044,Pasta en Conchas,Doria,250,1710.0,éxito,Pasta,g
1049,Pasta Cabello de Angel,Doria,250,1700.0,éxito,Pasta,g
1050,Pasta Cabello de Angel,Doria,500,3210.0,éxito,Pasta,g
1109,Pasta en Tornillos,Doria,250,1670.0,éxito,Pasta,g
1113,Pasta en Tornillos,Doria,250,2390.0,éxito,Pasta,g
1114,Pasta en Tornillos,Doria,500,3850.0,éxito,Pasta,g
1435,Pasta en Letras,Doria,250,1710.0,éxito,Pasta,g
1722,Pasta para Lasaña,Doria,200,3500.0,Ara,Pasta,g
